In [1]:
# Import Packages

import plotly.express as px
import torch as th
import pickle as pl
import pandas as pd
import numpy as np

/Users/magnusharder/Documents/UNI-DTU/6. Semester/Arkitektur - Special Course/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Load data
with open('Models/TransformerLoss.pkl', 'rb') as f:
    transformer_loss= pl.load(f)

# with open('Models/TransformerLoss.pkl', 'rb') as f:
#     transformer_loss, transformer_val_loss = pl.load(f)

with open('Models/RnnLoss.pkl', 'rb') as f:
    rnn_loss = pl.load(f)

time = np.linspace(0,1,2000)
df = pd.DataFrame({'Transformer Loss': transformer_loss, 'Time(H)': time})
df['Transformer Loss'] = df['Transformer Loss']/50


time2 = np.linspace(0,1,len(rnn_loss))
df2 = pd.DataFrame({'RNN Loss': rnn_loss, 'Time(H)': time2})
df2['RNN Loss'] = df2['RNN Loss']/50

print(len(transformer_loss))
print(len(rnn_loss))



2000
40000


In [91]:
#Plot data transformer vs rnn

fig = px.scatter()
fig.add_scatter(x=df['Time(H)'], y=df['Transformer Loss'],name="Transformer")
fig.add_scatter(x=df2['Time(H)'],y= df2['RNN Loss'],name="RNN")

fig.update_layout(legend=dict(title="Losses"))

fig.update_xaxes(
    title="Time(H)"
)

fig.update_yaxes(
    title="Training loss",type="log"
)
fig.show()



In [62]:
#Plot data RNN

fig = px.scatter(df,x='Time(H)', y='Transformer Loss')


fig.update_yaxes(
    type="log"
)

# fig.add_trace(transformer_val_loss)
fig.show()


NameError: name 'transformer_val_loss' is not defined

In [ ]:
#Plot data Transformer vs RNN

fig = px.scatter(df,x='Time(H)', y='Transformer Loss')
# fig.add_trace(transformer_val_loss)

fig.update_yaxes(
    type="log"
)
fig.show()


In [ ]:
## Testing model outputs:



#Load vocabularies
with open('data/English_encodings.pkl', 'rb') as f:
    english_encodings,english_sentences,Paddings_en,Vocab_en = pl.load(f)
with open('data/French_encodings.pkl', 'rb') as f:
    french_encodings,french_sentences,Paddings_fr,Vocab_fr = pl.load(f)

# Get the vocabulary size
src_vocab_size = Vocab_fr.__len__()
tgt_vocab_size = Vocab_en.__len__()
src_padding_idx = Vocab_fr.__getitem__('<Pad>')
tgt_padding_idx = Vocab_en.__getitem__('<Pad>')


# Load Train,Vaildation and Test data
with open('data/Train_data.pkl', 'rb') as f:
    X_train,Y_train,src_padding_mask,tgt_padding_mask = pl.load(f)

with open('data/Validation_data.pkl', 'rb') as f:
    X_vali,Y_vali,src_padding_mask_vali = pl.load(f)

with open('data/Test_data.pkl', 'rb') as f:
    X_test,Y_test,src_padding_mask_test = pl.load(f)

In [ ]:
import torch as th
from torchtext import vocab
import pickle as pl
from Models.Transformer import Transformer
from tqdm import tqdm
import numpy as np


# Intialize the model with set hyperparameters
T = 27
d_model = 512 # Dimension of the model (Embedding size)
d_ff = 2048 # Dimension of the feedforward network model in transformer
nhead = 8 # Number of heads in the multiheadattention models
dk = d_model//nhead
dv = d_model//nhead
num_layers = 6

# Initialize the model
Model = Transformer(
    T = T,
    d_model = d_model,
    nhead = nhead,
    d_ff = d_ff,
    dk = dk,
    dv = dv,
    num_layers = num_layers,
    src_vocab_size = src_vocab_size,
    tgt_vocab_size = tgt_vocab_size,
    src_padding_idx = src_padding_idx,
    tgt_padding_idx = tgt_padding_idx,
    dropout=0.1
)

Model.load_state_dict(torch.load('Models/Transformer.pt'))
Model.eval()
